# Модуль 3.3 — Память агента и безопасность

**Цель:** понять, как хранить контекст диалога и безопасно управлять доступом агента.

**Что сделаем:**
- установим библиотеки
- настроим ключ и base URL
- покажем память `ConversationBufferWindowMemory`
- разберем базовые правила безопасности

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Память агента: зачем и как

Память нужна, чтобы агент «помнил» контекст диалога:
- имя пользователя
- прошлые предпочтения
- уже сделанные шаги

Мы используем `ConversationBufferWindowMemory`, которая хранит **последние N сообщений**.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

base_url = os.environ.get("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=base_url,
)

memory = ConversationBufferWindowMemory(k=3, return_messages=True)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

print(conversation.predict(input="Привет! Меня зовут Нина, я учусь работать с LangChain."))
print(conversation.predict(input="Напомни, как меня зовут?"))
print(conversation.predict(input="Я люблю краткие ответы. Учти это."))
print(conversation.predict(input="Сделай краткое резюме нашего диалога."))

## Безопасность агента: базовые правила

**Основные риски:**
- утечка данных (секреты в промптах)
- выполнение опасных действий инструментами
- prompt injection (инструкции от пользователя, которые ломают правила)

**Практические рекомендации:**
- не давайте агенту прямой доступ к `os.system()` и файловой системе
- валидируйте входы инструментов через `args_schema`
- ограничивайте окно памяти (не храните лишнее)
- логируйте действия агента и вызовы инструментов
- явно задавайте правила в `system` сообщении